In [1]:
!ls

Untitled.ipynb   events-0.json    events-2.json    events-4.json
download.sh      events-1.json    events-3.json    events-mind.json


In [2]:
!pip3 install -q matplotlib


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [45]:
import json
import pprint
from collections import defaultdict
from dateutil.parser import parse

In [52]:
i = 0
lambdas = 0
non_lambdas = 0
cant_parse = 0

# ("year,day_of_year")

days = defaultdict(int)

for node in ("0"): # each node receives all 'Create' events
    for line in open(f"events-{node}.json"):
        i += 1
        try:
            j = json.loads(line)
        except Exception as e:
            cant_parse += 1

        date = j["time"].split("T")[0]
        if j["Event"]["EventName"] == "Created":
            spec = j["Event"]["Spec"]["Docker"].get("Entrypoint", [])
            is_lambda = "λ" in "".join(spec)
            # print(f"{spec} is lambda: {is_lambda}")
            if is_lambda:
                lambdas += 1
            else:
                non_lambdas += 1
                days[date] += 1

print(f"lambdas: {lambdas}, non-lambda jobs: {non_lambdas}, can't parse: {cant_parse}")

pprint.pprint(days)

lambdas: 154114, non-lambda jobs: 23554, can't parse: 4
defaultdict(<class 'int'>,
            {'2022-10-27': 143,
             '2022-10-28': 2101,
             '2022-10-29': 135,
             '2022-10-30': 105,
             '2022-10-31': 91,
             '2022-11-01': 674,
             '2022-11-02': 2060,
             '2022-11-03': 478,
             '2022-11-04': 51,
             '2022-11-05': 57,
             '2022-11-06': 37,
             '2022-11-07': 55,
             '2022-11-08': 14,
             '2022-11-09': 19,
             '2022-11-10': 58,
             '2022-11-11': 193,
             '2022-11-12': 12,
             '2022-11-13': 55,
             '2022-11-14': 28,
             '2022-11-15': 108,
             '2022-11-16': 196,
             '2022-11-17': 22,
             '2022-11-18': 1623,
             '2022-11-19': 305,
             '2022-11-20': 170,
             '2022-11-21': 852,
             '2022-11-22': 614,
             '2022-11-23': 129,
             '2022-11-24': 150

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline


ModuleNotFoundError: No module named 'matplotlib'

In [41]:
!tail -n 1 events-0.json

{"Type":"model.JobEvent","Event":{"JobID":"a42325c8-9eb8-479a-9571-425044c799c8","SourceNodeID":"QmVAb7r2pKWCuyLpYWoZr9syhhFnTWeFaByHdb8PkkhLQG","EventName":"ResultsPublished","Spec":{"Docker":{},"Language":{"JobContext":{}},"Wasm":{},"Resources":{"GPU":""},"Sharding":{}},"JobExecutionPlan":{},"Deal":{},"VerificationResult":{},"PublishedResult":{"StorageSource":"IPFS","Name":"job-a42325c8-9eb8-479a-9571-425044c799c8-shard-0-host-QmVAb7r2pKWCuyLpYWoZr9syhhFnTWeFaByHdb8PkkhLQG","CID":"QmPN8BmQLgCK9aUAfKbS3ZQ7EA2c5CNAdtc4JK3FpjttyT"},"EventTime":"2022-12-06T11:08:37.633481231Z","SenderPublicKey":"CAASpgIwggEiMA0GCSqGSIb3DQEBAQUAA4IBDwAwggEKAoIBAQCxeSv2y9/YrjGn0TOAHJ4ughLt40ty+ct13WLnnm5tiIGKKJVskbokj3/db7frMJ4CQaDlJLCyiecroQpMVKAIKviQ87oQ7kB9cnYAcBVnSCTucxMqQ1E0r4cWx5L97UpkNcQjMySM6fFX/93Yu/iNWP7/n4gL5FVXxEMBxxMQQa2efvNEw6VG7gy5ghbZCVcvesywTxL0TRPoptSEV/avykXmcTZ/IPJs1fm8AfJ4P6N+H7T7ZkhbetTdOupc3vAYwHP+Wh4EmPcxbv1zgSm6KdymECQG1B386GR7livdW/74oUhwyYylixX5nEy73Tq+DKi3q27e3OutcMXYmMTTAgMBAAE

In [40]:
!cat download.sh

#!/bin/bash
set -xeuo pipefail
gcloud compute scp bacalhau-vm-production-0:/data/.bacalhau/bacalhau-event-tracer.json --project bacalhau-production ./events-0.json &
gcloud compute scp bacalhau-vm-production-1:/data/.bacalhau/bacalhau-event-tracer.json --project bacalhau-production ./events-1.json &
gcloud compute scp bacalhau-vm-production-2:/data/.bacalhau/bacalhau-event-tracer.json --project bacalhau-production ./events-2.json &
gcloud compute scp bacalhau-vm-production-3:/data/.bacalhau/bacalhau-event-tracer.json --project bacalhau-production ./events-3.json &
gcloud compute scp bacalhau-vm-production-4:/data/.bacalhau/bacalhau-event-tracer.json --project bacalhau-production --zone europe-west4-a ./events-4.json &
scp luke@mind.lukemarsden.net:.bacalhau/bacalhau-event-tracer.json ./events-mind.json
